# **Chapter 05. Create Dataset**

* 基本上, 任何能夠以 `4D tensor (batch × height × width × channel)` 的形式作為輸入的資料都可以應用於電腦視覺的訓練方式！

---
##### 影像:
* 影像解析度 與 運算／儲存／傳輸所需資源 的 trade-off, 建議 `選擇壓縮格式(例如JPEG)、較高閾值(95%+)、較低解析度(取決於任務精細程度)`
* 通常影像會有3個channel(RGB), 但有些影像會有4個channel(RGBA), 其中A是 `alpha(透明度)`
* 常規影像處理流程: `影像以壓縮字串的形式讀取, 轉換為 3D uint8 tensor, 再將 [0, 255] 的像素值轉換為 [0, 1] 的浮點數`
* 現代最常用的排序是 [height, width, channel] 的順序, 稱為 channel-last 表達法, 例如 TensorFlow 就是如此 (早期則是 channel-first)

##### 地理空間資料:
* 從地圖產生的地理空間資料, 通常有可以被視為頻道的`柵格頻帶(raster band)`, 也就是具有特定特徵的像素(或更大的網格)被標註
* 例如: 圖片中的河流所在的網格, 像素值會被設定為1; 或是有15個州的地圖, 會產生15個頻道, 每個頻道各自標出一個州所在的像素

#### 音訊(audio)與視訊(video):
* 音訊是 1D 信號, 視訊則是 3D。 通常會建議採用專為他們設計的 ML 技術, 但其實也能夠利用處理影像的 ML 方法來簡單處理他們
* 音訊方面, 可以用 time-domain 或 frequency-domain 的資料當作輸入, 或是也可以用 NLP 的方式來處理它
* 視訊方面, 可以用 Conv3D 或是 其與RNN的結合來處理

---
### 手動標記標籤
* 可以`將各個類別都建立資料夾`, 再把對應的圖片放到資料夾中; 或是利用單一`CSV檔記錄下所有資料的 URL + label`
* 但 當圖片有不只一項 label 時, 就只能使用後者的方法了
* 大規模資料需要標記時, 可以使用 Computer Vision Annotation Tool 這個軟體來達成, 他提供了一個很好用於標記的UI介面
* 或是當一張圖片會有多項 label 時, 也可以使用 Jupyter Notebook 的互動功能來標記 (python 的 multi-label-pigeon)

##### Python: multi-label-pigeon

In [ ]:
%%bash
mkdir flower_images
for filename in 100080576_f52e8ee070_n.jpg 10140303196_b88d3d6cec.jpg 10172379554_b296050f82_n.jpg; do
  gsutil cp gs://practical-ml-vision-book-data/flowers_5_jpeg/flower_photos/daisy/$filename flower_images
done

In [3]:
import glob
filenames = glob.glob('flower_images/*.jpg')
print(filenames)

from multi_label_pigeon import multi_label_annotate
from IPython.display import display, Image

annotations = multi_label_annotate(
    filenames,
    options={'flower':['daisy','tulip', 'rose'], 'color':['yellow','red', 'other'],'location':['indoors','outdoors']},
    display_fn=lambda filename: display(Image(filename))
    )

['flower_images/10140303196_b88d3d6cec.jpg', 'flower_images/10172379554_b296050f82_n.jpg', 'flower_images/100080576_f52e8ee070_n.jpg']


HTML(value='0 examples annotated, 4 examples left')

flower


color


location


Output()

In [4]:
print(annotations)

{'flower_images/10140303196_b88d3d6cec.jpg': {'flower': ['daisy', 'daisy'], 'color': ['yellow', 'yellow'], 'location': ['indoors', 'outdoors', 'outdoors']}, 'flower_images/10172379554_b296050f82_n.jpg': {'flower': ['tulip'], 'color': ['red'], 'location': ['outdoors']}, 'flower_images/100080576_f52e8ee070_n.jpg': {'flower': ['daisy'], 'color': ['yellow'], 'location': ['outdoors', 'indoors']}}


### 自動標記標籤
* `Noisy Student`: 
    * 先人工手動標記一部份影像, 再利用這些影像訓練出 teacher model 並讓他預測更多的標籤
    * 再將兩者的組合當作訓練資料, 並利用 dropout 與 data augmentation 去訓練出 student model
    * 把表現得更好的 student model 更新成為新的 teacher, 不斷迭代訓練下去; 人工也可以在這時手動標記那些被模型視為預測信心值不高的影像
* `Self-Supervised Learning`:
    * 有時候在取得圖片時, 還無法立即得知他的標籤 (例如: 診斷前一段時間拍攝的醫學影像, 下雨或打雷前的氣象圖...)
    * 所以, 很多時候拍攝的當下無法標記出的影像, 也可能是值得保留的！


---
### Bias (偏見)
* 一個 `biased` 的資料集, 代表 `某些範例在資料集中的代表性不足或過多`, 以至於在遇到這些場景時準確率下降
* bias 的三個來源:
    1. `Selection Bias`: 訓練的資料是這個場景的偏斜子集合(skewed), 原因可能為:
        * 某些類型的資料取得比其他類型容易很多, 例如法國/義大利畫家畫像 vs. 斐濟/牙買加畫家畫像
        * 訓練資料是在較短的時間範圍內取得的, 例如晴天 vs 各種天候
        * 不恰當的資料清理, 例如貝殼資料集中拋棄出現動物的照片, 會導致甲殼類辨識的結果不佳
    2. `Measurement Bias`: 蒐集影像的方式在訓練與測試資料中不同, 例如:
        * 訓練資料以較高畫質的相機拍攝, 而測試資料來自畫質較低的攝影機
        * 狗與狼的分類器中, 如果狗的照片大多都在草地上拍攝, 而狼的照片大多來自於雪地, 模型可能學習到的是草和雪的分類特徵
    3. `Confirmation Bias`: 現實生活中值的分佈會導致模型強化不想要的部分, 例如:
        * 現實生活中, 消防員通常為女性, 這可能導致女性消防員的圖片被認為是cosplay的假消防員
        * 新聞過度強調某些族群的犯罪記錄, 會導致族群與犯罪有不預期的掛鉤
* biased $\neq$ imbalanced ! 不平衡可以是正常的現實資料分佈, 只要他不會讓模型以不想要的方式表現資料集的任何層面


---
### **建立資料集**
* 常見的 training/validation/testing 比例可以是 80:10:10, 或是只需要 training/validation 時 80:20
* 或是在需要 `交叉驗證(cross-validation)`時, 將 10% 固定為 testing, 其餘隨機分割為 training/validation 來多次訓練模型 -> 常用於小型資料集的訓練
* 注意: 不要在每一次訓練前重新拆分這三組資料, 否則會讓準確率無法被拿來比較

#### TensorFlow Record (TFRecord)
* TFRecord 是推薦的資料格式, 它是有 image & label 兩個 key 的字典, 也可以嵌入更多的 metadata (例如 bounding box)

In [17]:
import pandas as pd
df = pd.read_csv('gs://practical-ml-vision-book-data/flowers_5_jpeg/flower_photos/all_data.csv', names=['image','label'])

import numpy as np
np.random.seed(10)
rnd = np.random.rand(len(df))
train = df[ rnd < 0.8  ]
valid = df[ (rnd >= 0.8) & (rnd < 0.9) ]
test  = df[ rnd >= 0.9 ]
print(f"Total images: {len(df)}; Training: {len(train)}, Validation: {len(valid)}, Testing: {len(test)}")

Total images: 3670; Training: 2930, Validation: 359, Testing: 381


In [19]:
train.to_csv('TFRecord/train.csv', header=False, index=False)
valid.to_csv('TFRecord/valid.csv', header=False, index=False)
test.to_csv('TFRecord/test.csv', header=False, index=False)

outdf = test.head()     ## len(outdf) == 5
outdf.values

array([['gs://practical-ml-vision-book-data/flowers_5_jpeg/flower_photos/daisy/10466290366_cc72e33532.jpg',
        'daisy'],
       ['gs://practical-ml-vision-book-data/flowers_5_jpeg/flower_photos/daisy/10712722853_5632165b04.jpg',
        'daisy'],
       ['gs://practical-ml-vision-book-data/flowers_5_jpeg/flower_photos/daisy/11642632_1e7627a2cc.jpg',
        'daisy'],
       ['gs://practical-ml-vision-book-data/flowers_5_jpeg/flower_photos/daisy/13583238844_573df2de8e_m.jpg',
        'daisy'],
       ['gs://practical-ml-vision-book-data/flowers_5_jpeg/flower_photos/daisy/1374193928_a52320eafa.jpg',
        'daisy']], dtype=object)

In [ ]:
import tensorflow as tf
with tf.io.gfile.GFile('gs://practical-ml-vision-book-data/flowers_5_jpeg/flower_photos/dict.txt', 'r') as f:
    LABELS = [line.rstrip() for line in f]
print('Total Labels: {} , including {}, {}, {}, {}, and {}'.format(
    len(LABELS), LABELS[0], LABELS[1], LABELS[2], LABELS[3], LABELS[4]))

## --------   把資料轉成 TFRecord 的格式   --------
import apache_beam as beam
import tensorflow as tf

def _string_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value.encode('utf-8')]))

def _int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=value))

def _float_feature(value):
    return tf.train.Feature(float_list=tf.train.FloatList(value=value))

def read_and_decode(filename):
    IMG_CHANNELS = 3
    img = tf.io.read_file(filename)
    img = tf.image.decode_jpeg(img, channels=IMG_CHANNELS)
    img = tf.image.convert_image_dtype(img, tf.float32)
    return img

def create_tfrecord(filename, label, label_int):
    print(filename)
    img = read_and_decode(filename)
    dims = img.shape
    img = tf.reshape(img, [-1]) # flatten to 1D array
    return tf.train.Example(features=tf.train.Features(feature={
        'image': _float_feature(img),
        'shape': _int64_feature([dims[0], dims[1], dims[2]]),
        'label': _string_feature(label),
        'label_int': _int64_feature([label_int])
    })).SerializeToString()

with beam.Pipeline() as p:
    (p 
     | 'input_df' >> beam.Create(outdf.values)
     | 'create_tfrecord' >> beam.Map(lambda x: create_tfrecord(x[0], x[1], LABELS.index(x[1])))
     | 'write' >> beam.io.tfrecordio.WriteToTFRecord('TFRecord/output/train')
    )

Total Labels: 5 , including daisy, dandelion, roses, sunflowers, and tulips


gs://practical-ml-vision-book-data/flowers_5_jpeg/flower_photos/daisy/10466290366_cc72e33532.jpg
gs://practical-ml-vision-book-data/flowers_5_jpeg/flower_photos/daisy/10712722853_5632165b04.jpg
gs://practical-ml-vision-book-data/flowers_5_jpeg/flower_photos/daisy/11642632_1e7627a2cc.jpg
gs://practical-ml-vision-book-data/flowers_5_jpeg/flower_photos/daisy/13583238844_573df2de8e_m.jpg
gs://practical-ml-vision-book-data/flowers_5_jpeg/flower_photos/daisy/1374193928_a52320eafa.jpg


In [22]:
## splitting in Apache Beam
def hardcoded(x, desired_split):
    split, rec = x
    print('hardcoded: ', split, rec, desired_split, split == desired_split)
    if split == desired_split:
        yield rec

with beam.Pipeline() as p:
        splits = (p
                  | 'input_df' >> beam.Create([
                      ('train', 'a'),
                      ('train', 'b'),
                      ('valid', 'c'),
                      ('valid', 'd')
                  ]))
        
        split = 'train'
        _ = (splits
                 | 'h_only_{}'.format(split) >> beam.FlatMap(
                     lambda x: hardcoded(x, 'train'))
         )        
        split = 'valid'
        _ = (splits
                 | 'h_only_{}'.format(split) >> beam.FlatMap(
                     lambda x: hardcoded(x, 'valid'))
        )

hardcoded:  train a train True
hardcoded:  train a valid False
hardcoded:  train b train True
hardcoded:  train b valid False
hardcoded:  valid c train False
hardcoded:  valid c valid True
hardcoded:  valid d train False
hardcoded:  valid d valid True
